In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualization_utils as vis_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix,MeshcatVisualizer, StartMeshcat, Sphere
import pydrake.symbolic as sym
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver
from pydrake.all import PointCloud, MeshcatVisualizerParams, Role, HalfSpace, SceneGraph,CoulombFriction, Box, Rgba, WeldJoint, GeometrySet
from pydrake.polynomial import Polynomial as PolynomialCommon
import time
from scipy.spatial.transform import Rotation as sp_rot
from sampling_based_motion_planners import StraightLineCollisionChecker, PRM, BiRRT, PRMFixedEdges


from pydrake.all import (RationalForwardKinematics, ModelInstanceIndex,
                         SpatialInertia, RevoluteJoint, FixedOffsetFrame, MultibodyPlant)
from pydrake.geometry.optimization import HPolyhedron, Hyperellipsoid
from pydrake.geometry.optimization_dev import CspaceFreePath
from dataclasses import dataclass


In [4]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
meshcat = StartMeshcat()
meshcat.Delete()

DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile mechanism = RUNFILES_{MANIFEST_FILE,DIR}
DEBUG:drake:cwd = "/home/amice/Documents/coding_projects/drake/C_Iris_Examples"
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/n_link_pend_plane_viz.runfiles/drake/.drake-find_resource-sentinel' (good)
DEBUG:drake:FindRunfile found by-manifest '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/geometry/meshcat.js' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/n_link_pend_plane_viz.runfiles/drake/geometry/meshcat.js' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT

In [6]:

def AddPendulumLink(
    plant,
    pendulum_model_instance,
    link_dimensions,
    parent_frame,
    link_index: int,
    pend_axis = (0,0,1),
    color=np.array([1, 0, 0, 1]),
):
    pendulum_box = Box(*link_dimensions)
    pendulum_inertia = SpatialInertia.SolidBoxWithDensity(1.0, *link_dimensions)

    link_name = f"link_{link_index}"
    pend_body = plant.AddRigidBody(link_name, pendulum_model_instance, pendulum_inertia)
    parent_to_child_attach_frame = plant.AddFrame(
        FixedOffsetFrame(
            f"{link_name}_joint_frame",
            pend_body.body_frame(),
            RigidTransform(p=(0, link_dimensions[1] / 2, 0)),
        )
    )
    next_frame = plant.AddFrame(
        FixedOffsetFrame(
            f"{link_name}_joint_frame_next",
            pend_body.body_frame(),
            RigidTransform(p=(0, -link_dimensions[1] / 2, 0)),
        )
    )
    # add the revolute joint,
    joint_lim = np.pi
    plant.AddJoint(
        RevoluteJoint(
            f"joint_{link_index}",
            parent_frame,
            parent_to_child_attach_frame,
            pend_axis,
            -joint_lim,
            joint_lim,
        )
    )

    geom_id = plant.RegisterCollisionGeometry(
        pend_body, RigidTransform(), pendulum_box, link_name, CoulombFriction()
    )
    plant.RegisterVisualGeometry(
        pend_body, RigidTransform(), pendulum_box, link_name, color
    )

    return next_frame, geom_id


def N_Link_Pendulum(
    n: int,
    plant: MultibodyPlant,
    pendulum_model_instance: ModelInstanceIndex,
    link_dimensions: np.ndarray,
):
    assert n > 0
    parent_frame = plant.world_frame()
    colors = np.array(vis_utils.n_colors(n)) / 255
    colors = np.hstack([colors, np.ones((colors.shape[0], 1))])
    geom_ids = []
    for i in range(n):
        parent_frame, geom_id = AddPendulumLink(
            plant,
            pendulum_model_instance,
            link_dimensions,
            parent_frame,
            i + 1,
            [0 if i %2 == 0 else 1, 0, 1 if i%2 ==0 else 0],
            colors[i],
        )
        geom_ids.append(geom_id)
    return geom_ids


def AddRandomBox(
    plant: MultibodyPlant,
    obstacle_model_instance: ModelInstanceIndex,
    index: int,
    pos_limits,
    size_limits,
    color=[0, 0, 0, 1],
):
    link_name = f"obstacle_{index}"
    
    
    def random_center():
        dist = pos_limits[1] * np.sqrt(np.random.uniform(0,1))
        ang = np.random.uniform(0,2*np.pi)
        return dist*np.array([np.cos(ang), np.sin(ang), 0])
    center = random_center()#np.append(np.random.uniform(pos_limits[0], pos_limits[1], 2), 0)

    

    dimensions = np.append(np.random.uniform(size_limits[0], size_limits[1], 2), 1)
    box = Box(*dimensions)
    inertia = SpatialInertia.SolidBoxWithDensity(1, *dimensions)

    body = plant.AddRigidBody(link_name, obstacle_model_instance, inertia)
    
    random_rp = np.zeros(3)
    lim = np.pi / 10
    random_rp[2] = np.random.uniform(-lim, lim)
    angle = RotationMatrix(RollPitchYaw(random_rp))
    pose = RigidTransform(p=center) @ RigidTransform(R=angle)
    origin_local = pose.inverse()@RigidTransform()
    while np.all([-dimensions[i]/2 <= origin_local.translation()[i] < dimensions[i]/2 for i in range(3)]): 
        random_rp[2] = np.random.uniform(-lim, lim)
        angle = RotationMatrix(RollPitchYaw(random_rp))
        center = random_center()
        pose = RigidTransform(p=center)@RigidTransform(R=angle)
        origin_local = pose.inverse()@RigidTransform()
        
    plant.AddJoint(WeldJoint(link_name, plant.world_frame(), body.body_frame(), pose))
    col_id = plant.RegisterCollisionGeometry(
        body, RigidTransform(), box, link_name, CoulombFriction()
    )
    plant.RegisterVisualGeometry(body, RigidTransform(), box, link_name, color)
    return col_id

def AddNRandomBoxes(
    n: int,
    plant: MultibodyPlant,
    obstacle_model_instance: ModelInstanceIndex,
    pos_limits=(-10, 10),
    size_limits=(0.1, 1),
    color=[0, 0, 0, 1],
):
    assert n > 0
    geom_ids = []
    for i in range(n):
        geom_ids.append(AddRandomBox(plant, obstacle_model_instance, i, pos_limits, size_limits, color))
    return geom_ids


def make_line_polys(plant: MultibodyPlant, prm):
    polys = np.empty(shape=(plant.num_positions(), len(prm.prm.edges())), dtype=object)
    for i, (s0, s1) in enumerate(prm.prm.edges()):
        for j in range(plant.num_positions()):
            polys[j, i] = PolynomialCommon(np.array([s0[j], s1[j] - s0[j]]))
    return polys


## The plant parameters

In [7]:
meshcat.Delete()
num_pend = 3
plane_order = 1
maximum_path_degree = 1
num_obstacles = 10
exclude_pend_self_collisions = True 

## Construct the main plant we will do certification with

In [8]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

link_dimensions = (0.1, 0.3, 0.1)
pendulum_model_instance = plant.AddModelInstance("pendulum")
obstacle_model_instance = plant.AddModelInstance("obstacle")

np.random.seed(num_pend * num_obstacles)

pend_geom_ids = N_Link_Pendulum(
    num_pend, plant, pendulum_model_instance, np.array(link_dimensions)
)
pend_geom_set = GeometrySet(pend_geom_ids)

if num_obstacles > 0:
    obstacle_pos_limits = link_dimensions[1] * num_pend * np.array([-1.25, 1.25])
    obstacle_size_limits = (0.01 / np.log(num_obstacles), 0.5 / np.log(num_obstacles))
    obstacle_geom_ids=AddNRandomBoxes(
        num_obstacles,
        plant,
        obstacle_model_instance,
        obstacle_pos_limits,
        obstacle_size_limits,
    )
    obstacle_geom_set = GeometrySet(obstacle_geom_ids)
else:
    obstacle_geom_ids = None
    obstacle_geom_set = None

if exclude_pend_self_collisions:
    collision_filter_manager = scene_graph.collision_filter_manager()
    decl = CollisionFilterDeclaration().ExcludeWithin(pend_geom_set)
    collision_filter_manager.Apply(decl)

if meshcat is not None:
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    
plant.Finalize()



## Now add visualization geometries for the the "important bodies"
 
This constructs one plant for each important body purely for plotting hyperplanes


In [9]:
important_pend_col_geom_ids = [
    pend_geom_ids[-1]
]
important_obstacles_geom_ids = obstacle_geom_ids
important_collision_pairs = [(p1, p2) for p2 in important_obstacles_geom_ids 
                             for p1 in important_pend_col_geom_ids]

hyperplane_plants = {} # map a geometry pair to the appropriate hyerplane plant
plane_dimensions = (1,1,0.001)
plane_color = (0,1,0,0.25)
dummy_inertia = SpatialInertia.SolidBoxWithDensity(0.01, *plane_dimensions)
dummy_box = Box(*plane_dimensions)

for p1, p2 in important_collision_pairs:
    hyperplane_plant = builder.AddSystem(MultibodyPlant(0.01))
    hyperplane_plant.RegisterAsSourceForSceneGraph(scene_graph)
    builder.Connect(hyperplane_plant.get_geometry_poses_output_port(), scene_graph.get_source_pose_port(
              hyperplane_plant.get_source_id()))
    builder.Connect(
          scene_graph.get_query_output_port(),
          hyperplane_plant.get_geometry_query_input_port())
    hyperplane_model_instance = hyperplane_plant.AddModelInstance("hyperplane")
    body_name = f"hyperplane_{p1, p2}"
    body = hyperplane_plant.AddRigidBody(body_name, hyperplane_model_instance, dummy_inertia)    
    hyperplane_plant.RegisterVisualGeometry(body, RigidTransform(), dummy_box, body_name, plane_color)
    hyperplane_plant.Finalize()
    hyperplane_plants[p1, p2] = hyperplane_plant


## Build our diagram and our line certifier

In [10]:
diagram = builder.Build()

q_star = np.zeros(plant.num_positions())

if maximum_path_degree > 0:
    t0 = time.time()
    cspace_free_path = CspaceFreePath(
        plant, scene_graph, q_star, 
        maximum_path_degree=maximum_path_degree, 
        plane_order=plane_order
    )
    t1 = time.time()
    print(f"Time to construct line certifier for {num_pend}-links, {num_obstacles}-boxes = {t1 - t0}s")
else:
    print("Did not build cspace free path")
    self.cspace_free_path = None 

diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
hyperplane_plant_contexts = {k: diagram.GetMutableSubsystemContext(v, diagram_context) for k,v in hyperplane_plants.items()}
diagram.ForcedPublish(diagram_context)

DEBUG:drake:Generating collision pairs
DEBUG:drake:Collision pairs generated in 9.3442e-05 seconds
DEBUG:drake:Generating separating planes
DEBUG:drake:Separating planes generated in 0.000233225 seconds
DEBUG:drake:Generating TC-space rationals
DEBUG:drake:TC-space rationals generated in 0.030589509 seconds
DEBUG:drake:Pre-allocating PSD
DEBUG:drake:PSD allocated in 0.00105067 seconds
DEBUG:drake:Expanding rationals and pre-allocating programs
DEBUG:drake:Path rationals and programs allocated in 0.030730372 seconds


Time to construct line certifier for 3-links, 10-boxes = 0.06761813163757324s


### A helper method for plotting a hyperplane based on the collision pair. Make sure that the passed collision pair is actually "important"

In [23]:
def set_hyperplane_transform_by_col_pair(p1, p2, X):
    key = (p1, p2) if (p1,p2) in hyperplane_plants.keys() else (p2, p1)
    plant = hyperplane_plants[key]
    plant_context = hyperplane_plant_contexts[key]
    p = X.translation()
    quat = X.rotation().ToQuaternion().wxyz()
    plant.SetPositions(plant_context, np.append(quat, p))
    
X = RigidTransform(
    R = RotationMatrix(RollPitchYaw(np.pi/4,0,0)),
    p = np.array([0,0,0])
)

p1,p2 = important_collision_pairs[0]
set_hyperplane_transform_by_col_pair(p1,p2,X)
diagram.ForcedPublish(diagram_context)

## Construct some helper methods and a PRM to test out certification

In [12]:
Ratfk = RationalForwardKinematics(plant)
diagram_col_context = diagram.CreateDefaultContext()
plant_col_context = diagram.GetMutableSubsystemContext(plant, diagram_col_context)
scene_graph_col_context = diagram.GetMutableSubsystemContext(
        scene_graph, diagram_col_context
    )
query_port = scene_graph.get_query_output_port()

def check_collision_q_by_query(q):
    if np.all(q >= plant.GetPositionLowerLimits()) and np.all(
        q <= plant.GetPositionUpperLimits()
    ):
        plant.SetPositions(plant_col_context, q)
        query_object = query_port.Eval(scene_graph_col_context)
        return 1 if query_object.HasCollisions() else 0
    else:
        return 1

def check_collision_s_by_query(s):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_query(q)

def sample_col_free_point():
    q = np.random.uniform(
        plant.GetPositionLowerLimits(), 
        plant.GetPositionUpperLimits()
    )
    s = Ratfk.ComputeSValue(q, q_star)
    return s
l = 100
collision_checker = StraightLineCollisionChecker(
                    check_collision_s_by_query, 100
                )
prm = PRMFixedEdges(
                sample_col_free_point, l, collision_checker, dist_thresh=100
            )
diagram_prm_context = diagram.CreateDefaultContext()
plant_prm_context = diagram.GetMutableSubsystemContext(plant, diagram_prm_context)
vis_bundle = vis_utils.VisualizationBundle(
    diagram, diagram_context, plant, plant_prm_context,
    Ratfk, meshcat, q_star
)
end_effector = plant.GetBodyByName(f"link_{num_pend}")

prm.draw_tree(vis_bundle, end_effector)

PRM has 10 nodes
PRM has 13 edges
PRM has 20 nodes
PRM has 38 edges
PRM has 30 nodes
PRM has 67 edges
PRM has 40 nodes
PRM has 100 edges
PRM has 40 nodes
PRM has 100 edges


## Now certify the PRM

In [13]:
def make_line_polys(prm, max_num_edges = -1):
    
    polys = np.empty(shape=(plant.num_positions(), 
                            len(prm.prm.edges()) if max_num_edges < 0 else max_num_edges), dtype = object)
    for i, (s0, s1) in enumerate(prm.prm.edges()):
        for j in range(plant.num_positions()):
            if max_num_edges > 0 and not i < max_num_edges:
                break
            polys[j,i] = PolynomialCommon(np.array([s0[j],s1[j]-s0[j]]))
        
    return polys

path_safe = make_line_polys(prm)
# path_unsafe = make_line_polys(PRM_bad)
    

In [14]:
## Construct a trajectory and navigate the PRM.

In [46]:
from pydrake.all import PiecewisePolynomial
from tqdm import tqdm
traj = PiecewisePolynomial(path_safe.T,
                           np.arange(path_safe.shape[1]-1))


num_edges_to_navigate = 2
diagram_context.SetTime(0)
diagram.ForcedPublish(diagram_context)
animation = visualizer.StartRecording()
for idx in tqdm(range(num_edges_to_navigate)):
    has_col = False
    for t in np.linspace(0,1,100):
        cur_time = idx + t
        s = traj.value(cur_time)
        q = Ratfk.ComputeQValue(s, q_star)
        plant.SetPositions(plant_context, q)
        if check_collision_q_by_query(q):
            print(f"COL at idx {idx}")
            has_col = True
        
        X = RigidTransform(
            R = RotationMatrix(RollPitchYaw(*np.random.uniform(0, 2*np.pi, 3))),
            p = np.random.uniform(-1,1,3)
        )
        set_hyperplane_transform_by_col_pair(*important_collision_pairs[0], X)
        diagram_context.SetTime(time.time())
        diagram.ForcedPublish(diagram_context)
        time.sleep(0.01)
visualizer.StopRecording()
visualizer.PublishRecording()

100%|███████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.05s/it]


In [45]:
animation.autoplay()


True

### Now certify

In [16]:
cert_options = CspaceFreePath.FindSeparationCertificateGivenPathOptions()
cert_options.terminate_segment_certification_at_failure = False

cert_options.num_threads = -1
cert_options.verbose = False
cert_options.solver_id = MosekSolver.id()
cert_options.solver_options = SolverOptions()
cert_options.terminate_path_certification_at_failure = False



In [17]:
t0 = time.time()
statistics, cert_result = \
    cspace_free_path.FindSeparationCertificateGivenPath(path_safe, 
                                                        set(),
                                                        cert_options)
t1 = time.time()

print(f"num pairs to certify {len(statistics[0].total_time_to_certify_pair)}")
print(f"Certification of safe PRM for {num_pend} links in {t1-t0}s")
print(f"Frac edges safe = {sum([1 if s.certified_safe() else 0 for s in statistics])}/{len(statistics)}")
print(f"Unsafe inds = {[idx for idx, s in enumerate(statistics) if not s.certified_safe()]}")


num pairs to certify 30
Certification of safe PRM for 3 links in 2.7259695529937744s
Frac edges safe = 94/100
Unsafe inds = [51, 54, 55, 58, 64, 66]


In [20]:
# cert result is indexed by the prm edge. 
first_edge_cert_result = cert_result[0]

desired_result_tuple = important_collision_pairs[0]
def get_associated_plane_result_tuple(p1, p2, cert_result_for_edge):
    key = (p1, p2) if (p1,p2) in cert_result_for_edge.keys() else (p2,p1)
    return cert_result_for_edge[key]
sep_cert_result = get_associated_plane_result_tuple(*desired_result_tuple,
                                                    first_edge_cert_result)

# Here is the plane
print(sep_cert_result.a)
print(sep_cert_result.b)

[<Polynomial "-1.2962070284903098*1 + 1.6513447557858365*mu">
 <Polynomial "-4.0257814963558811*1 + -2.1277392404648405*mu">
 <Polynomial "-0.00075917889830485054*1 + 0.21178809313777194*mu">]
1.7135451885758035*1 + 0.6383490565701222*mu


## Set up the sliders so we can move the plant around manually

In [31]:
sliders = []
for i in range(plant.num_positions()):
    sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=plant.GetPositions(plant_context)[i],
                                       description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(diagram_context)

    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

FloatSlider(value=1.7885314878249217, description='q0', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=-1.477444661627827, description='q1', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=-1.0092865513822509, description='q2', max=3.141592653589793, min=-3.141592653589793)